## Import required packages 

In [18]:
import pandas as pd 
import numpy as np

In [ ]:
import requests
import pandas as pd

# Replace 'YOUR_API_KEY' with your actual Polygon.io API key
api_key = 'BIf89HTb0NaGQsWozgDr5x1W8GfFC3MH'
symbol = 'AAPL'
url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/2020-01-01/2023-01-01?apiKey={api_key}'

response = requests.get(url)
data = response.json()

# Convert the JSON data to a pandas DataFrame
df = pd.DataFrame(data['results'])

# Convert the timestamp to a readable date format
df['t'] = pd.to_datetime(df['t'], unit='ms')

# Display the historical data
print(df)

KeyError: 'results'

In [26]:
import requests
import pandas as pd

api_key = 'BIf89HTb0NaGQsWozgDr5x1W8GfFC3MH'
symbol = 'AAPL'
url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/2023-01-01/2024-01-01?apiKey={api_key}'

response = requests.get(url)
data = response.json()

# Convert the JSON data to a pandas DataFrame
df = pd.DataFrame(data['results'])

# Convert the timestamp to a readable date format
df['t'] = pd.to_datetime(df['t'], unit='ms')

# Display the historical data
print(df)

              v        vw        o       c        h        l  \
0    58280413.0  134.9653  134.080  135.27  136.250  133.770   
1    80200655.0  136.3762  135.280  137.87  138.020  134.220   
2    81760313.0  141.2116  138.120  141.11  143.315  137.900   
3    66435142.0  142.0507  140.305  142.53  143.160  140.300   
4    65799349.0  140.7526  140.890  141.86  142.430  138.810   
..          ...       ...      ...     ...      ...      ...   
234  37149570.0  194.1013  195.180  193.60  195.410  192.970   
235  28919310.0  193.1713  193.610  193.05  193.890  192.830   
236  48087681.0  192.5679  192.490  193.15  193.500  191.090   
237  34049898.0  193.9222  194.140  193.58  194.660  193.170   
238  42672148.0  192.5761  193.900  192.53  194.400  191.725   

                      t       n  
0   2023-01-19 05:00:00  491674  
1   2023-01-20 05:00:00  552230  
2   2023-01-23 05:00:00  719288  
3   2023-01-24 05:00:00  498679  
4   2023-01-25 05:00:00  536505  
..                  ...    

In [25]:
data

{'status': 'NOT_AUTHORIZED',
 'request_id': '6a85e10a46fd447f7ae0e87866f46575',
 'message': "Your plan doesn't include this data timeframe. Please upgrade your plan at https://polygon.io/pricing"}